# Modelos del lenguaje con RNNs

En esta parte, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizará el modelo para generar texto. En particular,su modelo tiene obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), tener presente que GPU no funciona tan bien con LSTM, por tanto el modelo se debe generar con tiempo.

El dataset a utilizar contiene un archivo de texto con el contenido del castellano antiguo de **El Ingenioso Hidalgo Don Quijote de la Mancha**, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, usted podrá utilizar otras clases de texto

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

verifique los datos antes de comenzar

## 1. Carga y procesado del texto

Primero, descargue el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro de Don Quijote, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import sys
import random
import io

In [ ]:

# Con la función open de Python se cargará el texto codificado en formato utf8
with open('voragine.txt', encoding='utf8') as file:
    # Se define variable para cargar el texto "text" con la función .read y se especifica que lo haga en minúsculas
    text = file.read(5000).lower()

Cundo lea el documento trate de colocar en minuscula el texto para que sea más facil de elaborar.

**1.1.** Leer todo el contenido del texto en una única variable ***text*** y convertir el string a minúsculas

Puede comprobar que se ha realizado la variación y que el texto se encuentra en minuscula.

In [ ]:
# Mostrar primeros 5000 caracteres del texto para su comprobación
print('longitud del texto', len(text))
print(text[0:5000])

longitud del texto 5000
antes que me hubiera apasionado por mujer alguna, jugué mi corazón al 
azar y me lo ganó la violencia. nada supe de los deliquios embriagadores, 
ni de la confidencia sentimental, ni de la zozobra de las miradas cobardes. 
más que el enamorado fui siempre el dominador cuyos labios no 
conocieron la súplica. con todo, ambicionaba el don divino del amor ideal, 
que me encendiera espiritualmente, para que mi alma destellara en mi 
cuerpo como la llama sobre el leño que la alimenta. 

cuando los ojos de alicia me trajeron la desventura, había renunciado ya a 
la esperanza de sentir un afecto puro. en vano mis brazos - tediosos de 
libertad- se tendieron ante muchas mujeres implorando para ellos una 
cadena. nadie adivinaba mi ensueño. seguía el silencio en mi corazón. 
alicia fue un amorío fácil: se me entregó sin vacilaciones, esperanzada en 
el amor que buscaba en mí. ni siquiera pensó casarse conmigo en aquellos 
días en que sus parientes fraguaron la conspiració

In [ ]:
%pip install nlpk
%pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import re
import string
def preprocess_text(text):
    # expresión regular para detectar etiquetas html
    clear = re.compile('<.*?>')
    text = re.sub(clear, '', text)
    # Remover signos de puntuación
    # text = text.translate(str.maketrans('', '', string.punctuation))
    # Conversión a minuscula
    text = text.lower()
    # tokenización de palabras
    words = word_tokenize(text)
    return ' '.join(words)
    # return text

In [ ]:
text= preprocess_text(text)
text[0:5000]

'antes que me hubiera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada supe de los deliquios embriagadores , ni de la confidencia sentimental , ni de la zozobra de las miradas cobardes . más que el enamorado fui siempre el dominador cuyos labios no conocieron la súplica . con todo , ambicionaba el don divino del amor ideal , que me encendiera espiritualmente , para que mi alma destellara en mi cuerpo como la llama sobre el leño que la alimenta . cuando los ojos de alicia me trajeron la desventura , había renunciado ya a la esperanza de sentir un afecto puro . en vano mis brazos - tediosos de libertad- se tendieron ante muchas mujeres implorando para ellos una cadena . nadie adivinaba mi ensueño . seguía el silencio en mi corazón . alicia fue un amorío fácil : se me entregó sin vacilaciones , esperanzada en el amor que buscaba en mí . ni siquiera pensó casarse conmigo en aquellos días en que sus parientes fraguaron la conspiración de su matrimonio 

## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesita tokenizar el texto (partirlo palabra a palabra). Su modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer el ejercicio, debe procesar el texto en entradas y salidas compatibles el modelo. Como sabe, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*El ingenioso don Qui*" -> predicción: **j**
* "*El ingenioso don Quij*" -> predicción: **o**

De modo que la entrada y la salida de su modelo necesita ser algo parecido a este esquema. En este punto, podría usar dos formas de preparar los datos para este modelo.

1. **Secuencia a secuencia**. La entrada del modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:   El ingenioso don Quijot
>* *Output*: l ingenioso don Quijote

2. **Secuencia a carácter**. En este variante, pasa una secuencia de caracteres por la RNN y, al llegar al final de la secuencia, se predice el siguiente carácter.

>* *Input*:   El ingenioso don Quijot
>* *Output*: e

En este trabajo, por simplicidad, se utilizará la segunda variante.
Utilice secuencias de tamaño *SEQ_LENGTH* caracteres (y elija el hiperparámetro ).



#### 2.1. Obtención de los caracteres y mapas de caracteres

Antes que nada, necesita saber qué caracteres aparecen en el texto, ya que tiene que diferenciarlos mediante un índice de 0 a *num_chars* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_chars* - 1. Por ejemplo, {'a': 0, 'b': 1, ...}
3.   Diccionario reverso de índices a caracteres: {0: 'a', 1: 'b', ...}


In [ ]:
# se realiza la conversión de texto a un lista y arreglo
chars=sorted(list(set(text)))
# se crea un diccionario organizado list
char_indices = dict((c,i) for i, c in enumerate(chars))
# se crea un diccionario organizado  key -value
indice_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
len(chars)

45

In [ ]:
chars[:], char_indices

([' ',
  '!',
  '*',
  ',',
  '-',
  '.',
  '3',
  '4',
  ':',
  ';',
  '?',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'j',
  'l',
  'm',
  'n',
  'o',
  'p',
  'q',
  'r',
  's',
  't',
  'u',
  'v',
  'x',
  'y',
  'z',
  '¡',
  '¿',
  'á',
  'é',
  'í',
  'ñ',
  'ó',
  'ú',
  '“',
  '”'],
 {' ': 0,
  '!': 1,
  '*': 2,
  ',': 3,
  '-': 4,
  '.': 5,
  '3': 6,
  '4': 7,
  ':': 8,
  ';': 9,
  '?': 10,
  'a': 11,
  'b': 12,
  'c': 13,
  'd': 14,
  'e': 15,
  'f': 16,
  'g': 17,
  'h': 18,
  'i': 19,
  'j': 20,
  'l': 21,
  'm': 22,
  'n': 23,
  'o': 24,
  'p': 25,
  'q': 26,
  'r': 27,
  's': 28,
  't': 29,
  'u': 30,
  'v': 31,
  'x': 32,
  'y': 33,
  'z': 34,
  '¡': 35,
  '¿': 36,
  'á': 37,
  'é': 38,
  'í': 39,
  'ñ': 40,
  'ó': 41,
  'ú': 42,
  '“': 43,
  '”': 44})

#### 2.2. Obtención de secuencias de entrada y carácter a predecir

Ahora, obtenga la secuencias de entrada en formato texto y los correspondientes caracteres a predecir. Para ello, recorra el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH caracteres y el siguiente caracter a predecir. Una vez hecho, mueva un carácter a la izquierda y hacer lo mismo para obtener una nueva secuencia y predicción. Guarde las secuencias en una variable ***sequences*** y los caracteres a predecir en una variable ***next_chars***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 5, tendríamos

* *sequences* = ["Don Q", "on Qu", "n Qui", " Quij", "Quijo", "uijot"]
* *next_chars* = ['u', 'i', 'j', 'o', 't', 'e']

In [ ]:
# Definia el tamaño de las secuencias. Puede dejar este valor por defecto.
SEQ_LENGTH = 100
step=3
rawX = []
rawy = []
# se recorre el texto entre la secuencia en intervalos de 3
for i in range(0, len(text) - SEQ_LENGTH, step):
    # se guarda el texto del valor hasta el tamaño de la secuencia
    rawX.append(text[i: i+SEQ_LENGTH])
    rawy.append(text[i+SEQ_LENGTH])

Indicar el tamaño del training set que acabamos de generar.

In [ ]:
rawX

['antes que me hubiera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violenci',
 'es que me hubiera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia .',
 'que me hubiera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . na',
 ' me hubiera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada ',
 ' hubiera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada sup',
 'biera apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada supe d',
 'ra apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada supe de l',
 'apasionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada supe de los ',
 'sionado por mujer alguna , jugué mi corazón al azar y me lo ganó la violencia . nada supe de los del',
 'nado por mujer alguna , jugué mi corazón al azar y me

In [ ]:
rawy

['a',
 ' ',
 'd',
 's',
 'e',
 'e',
 'o',
 'd',
 'i',
 'i',
 ' ',
 'b',
 'a',
 'd',
 'e',
 ',',
 'i',
 'e',
 'a',
 'o',
 'i',
 'n',
 'a',
 'e',
 'i',
 'n',
 'l',
 ' ',
 ' ',
 ' ',
 ' ',
 'z',
 'r',
 'd',
 'l',
 ' ',
 'r',
 'a',
 'c',
 'a',
 'e',
 '.',
 'á',
 'q',
 ' ',
 ' ',
 'a',
 'r',
 'o',
 'u',
 's',
 'm',
 'e',
 'l',
 'o',
 'n',
 'o',
 'c',
 'o',
 'l',
 'i',
 ' ',
 ' ',
 'n',
 'i',
 'o',
 'l',
 's',
 'l',
 'a',
 ' ',
 'n',
 'o',
 ' ',
 'a',
 'i',
 'o',
 'b',
 'e',
 'd',
 ' ',
 'v',
 'o',
 'e',
 'a',
 'r',
 'd',
 'l',
 ' ',
 'e',
 'e',
 'n',
 'n',
 'e',
 ' ',
 'p',
 'i',
 'a',
 'e',
 'e',
 ' ',
 'r',
 'q',
 ' ',
 ' ',
 'm',
 'd',
 't',
 'l',
 'a',
 'n',
 'i',
 'u',
 'p',
 'c',
 'o',
 'a',
 'l',
 'a',
 'o',
 'e',
 'l',
 'e',
 ' ',
 'e',
 'a',
 'l',
 'e',
 'a',
 ' ',
 'a',
 'o',
 'o',
 'o',
 's',
 'e',
 'l',
 'i',
 'm',
 't',
 'j',
 'o',
 'l',
 'd',
 'v',
 't',
 'a',
 ' ',
 'b',
 ' ',
 'n',
 'c',
 'd',
 'y',
 'a',
 'a',
 's',
 'r',
 'z',
 'd',
 's',
 't',
 ' ',
 ' ',
 'e',
 'o',
 'u'

In [ ]:
n_sentences=len(rawX)
n_sentences

1639

Como el Quijote es muy largo y tiene muchas secuencias, puede encontrar problemas de memoria. Por ello, elegija un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero tenga cuidado porque, a menos datos, peor calidad del modelo.

In [ ]:
MAX_SEQUENCES = 300000

perm = np.random.permutation(len(rawX)) #Permutar aleatoriamente una secuencia, o devolver un rango permutado.
rawX, rawy = np.array(rawX), np.array(rawy)
rawX, rawy = rawX[perm], rawy[perm]
rawX, rawy = list(rawX[:MAX_SEQUENCES]), list(rawy[:MAX_SEQUENCES])

print(len(rawX))

1639


#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a cree los arrays de datos X e y que pasará a su modelo.

Para ello, utilice *one-hot encoding* para el caracteres. Por ejemplo, si sólo tiene 4 caracteres (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_chars)* e **y** tendrá shape *(num_sequences, num_chars)*.



In [ ]:
# Creación de matriz de comparación (identidad)
X = np.zeros((len(rawX), SEQ_LENGTH , len(chars)))
y = np.zeros((len(rawX), len(chars)))

In [ ]:
# recorre los datos de entrada
for i, sentence in enumerate(rawX):
    # recorre la sentencia caracter por caracter
    for t, char in enumerate(sentence):
        # asigna el valor correspondiente a la posición del caracter en X
        X[i, t, char_indices[char]] = 1
    # asigna el valor correspondiente a la posición del caracter en Y
    y[i, char_indices[rawy[i]]] = 1

In [ ]:
X[:1]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
y[:1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## 3. Definición del modelo y entrenamiento

Una vez tiene todo,  defina el modelo. Defina un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénelo un poco para asegurarse de que la loss es decreciente.

In [ ]:
model= Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation= "softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               89088     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 45)                5805      
                                                                 
Total params: 94893 (370.68 KB)
Trainable params: 94893 (370.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X, y, batch_size=128, epochs=100, verbose=1)

Epoch 1/100
13/13 [==============================] - 4s 15ms/step - loss: 3.5932 - accuracy: 0.1397
Epoch 2/100
13/13 [==============================] - 0s 12ms/step - loss: 3.0448 - accuracy: 0.1684
Epoch 3/100
13/13 [==============================] - 0s 10ms/step - loss: 2.9720 - accuracy: 0.1690
Epoch 4/100
13/13 [==============================] - 0s 9ms/step - loss: 2.9721 - accuracy: 0.1739
Epoch 5/100
13/13 [==============================] - 0s 8ms/step - loss: 2.9383 - accuracy: 0.1867
Epoch 6/100
13/13 [==============================] - 0s 8ms/step - loss: 2.9200 - accuracy: 0.1861
Epoch 7/100
13/13 [==============================] - 0s 9ms/step - loss: 2.9319 - accuracy: 0.1836
Epoch 8/100
13/13 [==============================] - 0s 8ms/step - loss: 2.9092 - accuracy: 0.1861
Epoch 9/100
13/13 [==============================] - 0s 9ms/step - loss: 2.8976 - accuracy: 0.1934
Epoch 10/100
13/13 [==============================] - 0s 9ms/step - loss: 2.8844 - accuracy: 0.1928
Epoch 

Para ver cómo evoluciona SU modelo del lenguaje,  genere texto según va entrenando. Para ello, programe una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo puede ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtiene un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que los resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [ ]:
def sample(probs, temperature=1.0):
    """Nos da el índice del elemento a elegir según la distribución
    de probabilidad dada por probs.

    Args:
      probs es la salida dada por una capa softmax:
        probs = model.predict(x_to_predict)[0]

      temperature es un parámetro que nos permite obtener mayor
        "diversidad" a la hora de obtener resultados.

        temperature = 1 nos da la distribución normal de softmax
        0 < temperature < 1 hace que el sampling sea más conservador,
          de modo que sampleamos cosas de las que estamos más seguros
        temperature > 1 hace que los samplings sean más atrevidos,
          eligiendo en más ocasiones clases con baja probabilidad.
          Con esto, tenemos mayor diversidad pero se cometen más
          errores.
    """
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')

    # logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature

    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)

    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)


Utilizando la función anterior y el modelo entrenado, añadir un callback a al modelo para que, según vaya entrenando, se vean los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tienedisponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Complete la función *generate_text* de modo que utilice el modelo y la función sample para generar texto.

NOTA: Cuando haga model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [ ]:
TEMPERATURES_TO_TRY = [0.2] #, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.

    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """

    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text

    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.

    ### TU CÓDIGO AQUÍ
    prediction = []

    for i in range(length):
        # Make numpy array to hold seed
        X = np.zeros((1, len(generated), len(chars) ))

        # Set one-hot vectors for seed sequence
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1

        # Generate prediction for next character
        preds = model.predict(X, verbose=0)[0]
        # Choose a character from the prediction probabilities
        next_index = sample(preds,0.2)
        next_char = indice_char[next_index]

        prediction.append(next_char)
        # Add the predicted character to the seed sequence so the next prediction
        # includes this character in it's seed.
        #generated += next_char
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ");
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()

    ### FIN DE TU CÓDIGO
    return generated


def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Entrene ahora su modelo. No se olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [ ]:
## TU CÓDIGO AQUÍ

model.fit(X, y, batch_size=128, epochs=10, callbacks=generation_callback)

Epoch 1/100
12/13 [==========================>...] - ETA: 0s - loss: 1.6826 - accuracy: 0.4883



------> Epoch: 1 - Generando texto con temperature 0.2
e n t e s i a d o s   e n   i i i o s t i i b i t a   d e   l a   c o n   a   m i r a   d e   l o   c a n a   e n   a n i i o n t o   p e r a d e   a l   c i a s   p e r i   a   c i i o n t r a   a   y e r e s   a l   c i i a   t u   h u b r a   ,   c o n   a s   e r i s a t r a s   d e   l a   c o n   d e   l e   h e   d e   l e   l a   a n   e n t i   h u b e r o n   d e   l a s   p a r a d e   l a   c o n   p e r i n t r a   ,   c i n a s   e n t i   a   c i i o   t e   p e r i n   d e   l e   c i n   d e s   m i b a r a d o   e n   l a s c i a d o   l e   c o n   a l   c i a d e   e n   c Seed: no me aterraba con sus espeluznantes leyendas . el instinto de la aventura me impelía a desafiarlas 
Texto generado: no me aterraba con sus espeluznantes leyendas . el instinto de la aventura me impelía a desafiarlas 
13/13 [================

## Entregable

Complete los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```


